In [3]:
from pandas import Series, DataFrame
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


<b> Q1. Read in the files gold.txt and labels.txt. </b> The gold DataFrame should have columns url and category, while the labels DataFrame should have columns turk, url and category. You will have to pick the right separator.

In [4]:
gold = pd.read_csv("gold.txt", sep = '\t', header = None, names = ['url','category'])
labels = pd.read_csv("labels.txt", sep = '\t', header = None, names = ['turks','url','category'])

In [5]:
gold[0:5]

,url,category
0,http://0800-horoscope.com,G
1,http://18games.net,X
2,http://1pixelout.net,G
3,http://1porno.com/004/teens/index.html,X
4,http://1stpussy.com/005/16/index.html,X


In [6]:
labels[:5]

,turks,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G


<B>Q2. Split into two DataFrames.</B> Split the labels DataFrame into two: labels on gold and labels unknown, the former containing all rows where the url is present in the gold set, and
the latter one contains all remaining rows of labels.

In [7]:
labels_on_gold = pd.merge(labels,gold,on = 'url',suffixes = ['_labels','_gold'])
labels_unknown = labels[~labels.url.isin(labels_on_gold.url)]

In [8]:
labels_on_gold[:5]

,turks,url,category_labels,category_gold
0,A1253FXHCZ9CWM,http://0800-horoscope.com,G,G
1,A153PKAL7OAY36,http://0800-horoscope.com,G,G
2,A1FV9SAPL5C6KY,http://0800-horoscope.com,G,G
3,A1JTOT0DWM6QGL,http://0800-horoscope.com,G,G
4,A1PXXEOGQ76RNJ,http://0800-horoscope.com,G,G


In [9]:
labels_unknown[:5]

,turks,url,category
0,A1OT3A29R9N1DG,http://000.cc,P
1,A1PXXEOGQ76RNJ,http://000.cc,G
2,A1PXXEOGQ76RNJ,http://000.cc,G
3,A21US576U8SCO4,http://000.cc,G
4,A2LGX47NN7C5D3,http://000.cc,G


<B>Q3. Compute accuracies of turks.</B> Create a rater goodness DataFrame that is indexed by turk, and has two columns: the number of ratings, and the average correctness of ratings for each turk (both on gold set urls).

In [10]:
def rating_diff(correct_label):
    if correct_label['category_labels'] == correct_label['category_gold']:
        correct = 1
    else:
        correct = 0
    return correct

In [11]:
rating_differences = labels_on_gold.T.apply(rating_diff)
labels_on_gold['same category'] = rating_differences
rater_goodness = labels_on_gold.groupby('turks')['same category'].agg(["count","mean"])
rater_goodness[:10]

,count,mean
turks,,
A112DVP1KG4QZU,1,1.000000
A1253FXHCZ9CWM,29,0.517241
A12CY1Q7XKJJDE,1,1.000000
A12RE8G66WTO8B,20,0.750000
A12Y1GTGIQDGRA,3,0.333333
A13CEW9JGDWGX1,1,1.000000
A13OE9GBRJ0S2U,4,0.750000
A14IQ4GLNWNPOJ,1,1.000000
A153PKAL7OAY36,148,0.722973


<B>Q4. Odds ratios.</B> If someone is correct p fraction of the time, the odds of success are defined as: odds = p/1.001 − p. Attach column called odds to the rater goodness DataFrame, using the average correctness of the turk as his or her p.

In [12]:
rater_goodness['odds'] = rater_goodness['mean']/(1.001 - rater_goodness['mean'])
rater_goodness[:10]

,count,mean,odds
turks,,,
A112DVP1KG4QZU,1,1.000000,1000.000000
A1253FXHCZ9CWM,29,0.517241,1.069214
A12CY1Q7XKJJDE,1,1.000000,1000.000000
A12RE8G66WTO8B,20,0.750000,2.988048
A12Y1GTGIQDGRA,3,0.333333,0.499251
A13CEW9JGDWGX1,1,1.000000,1000.000000
A13OE9GBRJ0S2U,4,0.750000,2.988048
A14IQ4GLNWNPOJ,1,1.000000,1000.000000
A153PKAL7OAY36,148,0.722973,2.600369


<B>Q5. Most accurate turks.</B> List the top 10 most accurate turks who have rated at least 20 gold set URLs.

In [13]:
mask = rater_goodness['count'] >= 20
rater_goodness[mask].sort_values('odds',ascending = False)[:10]

,count,mean,odds
turks,,,
A2U0R4X38GUKZE,20,0.950000,18.627451
A22C0PJUBFJTI0,36,0.916667,10.869565
A23YQUBXZPKILZ,24,0.875000,6.944444
ATVALOQVDCMZW,103,0.854369,5.826657
A1HIXWH4OXT8S4,40,0.825000,4.687500
A3220HG1O83HQ4,22,0.818182,4.475385
A32W20KGQXS0LL,25,0.800000,3.980100
A20PWAB7G3HDHU,20,0.800000,3.980100
AJSJVK40F5HM6,28,0.785714,3.649635


<B>Q6. Rating counts versus accuracy.</B> One may imagine that a committed and accurate turk will rate lots of URLs. On the other hand, perhaps it is only the spammers who have the time to rate lots of URLs. Is number of ratings by a turker on gold set URLs related to his or her accuracy? There’s no fixed answer; just try to show some evidence for your answer.

<B>Q7. Overall predicted odds. Consider each url u that is not in the gold set, and each category c.</B> For the pair (u, c), calculate the  product of odds of all turks who (a) rated url u as category c, and (b) have rated more gold set urls than 75% of all turks who rated at least one gold-set url. For example, if you find that there are 269 turks who rated at least one gold-set url, you want to select only the turks who have rated more gold-set urls than 75% of these 269 turks. We can think of these as our “reliable” turks. Now, our belief that url u belongs to category c depends on how many “reliable” turks rated u as c; specifically, our belief is based on the product of their reliability scores (i.e., their odds). We shall call such products of odds the overall odds henceforth.

In [25]:
s = rater_goodness['count'].quantile(q=0.75) 
mask = rater_goodness['count'] >= s
turks_temp = rater_goodness[mask] 
turks_reliable = turks_temp[['odds']].reset_index()

labels_unknown75 = pd.merge(turks_reliable,labels_unknown, on = 'turks')
labels_overall_odds = labels_unknown75.groupby(['url','category']).prod().reset_index().rename(columns = {'odds':'overall odds'})
labels_overall_odds.to_csv('over_odd.csv')
labels_overall_odds[:10]

,url,category,overall odds
0,http://0-101.net,G,2.155963
1,http://000.cc,G,1.460583
2,http://0000.jp,G,14.488244
3,http://000relationships.com,G,9.914590
4,http://000relationships.com,P,1.851852
5,http://000vitamins.com,G,3.784982
6,http://000webhost.com,G,11.159502
7,http://003my.com,G,4.912040
8,http://007absolutehardcore.com/early_cumshots/...,P,3.346522
9,http://007absolutehardcore.com/early_cumshots/...,R,12.290450


<B>Q8. Predicted categories.</B> Create a DataFrame (called result_75, whose index is URLs not in the gold set, and with two columns called top category and top odds. The top category should be the category with the highest overall odds for that url,
and the top odds should be the overall odds for that top category. These are our predictions, and the confidence we have in them (higher overall odds implies greater confidence). If you want, you can check to see if the predicted categories make sense.

In [26]:
top_category = 

NameError: name 'top_category' is not defined